For more analysis <br>
http://people.csail.mit.edu/indyk/6.838-old/handouts/lec17.pdf

In [11]:
import numpy as np
import pandas as pd

In [20]:
# below is 1D 
# assume all values are distinct

def closest_pair_split(arr1, arr2): # sorted arrays in advance with mergesort
    out = []
    delta = None
    c_pair = None
    i = 0
    j = 0
    
    while i < len(arr1) and j < len(arr2):
        delta_c = np.abs(arr1[i] - arr2[j])
        if delta == None or delta > delta_c:
            delta = delta_c
            c_pair = (arr1[i], arr2[j])
        
        if arr1[i] < arr2[j]:
            out.append(arr1[i])
            i += 1
        else:
            out.append(arr2[j])
            j += 1
    
    while i < len(arr1):
        out.append(arr1[i])
        i += 1
    
    while j < len(arr2):
        out.append(arr2[j])
        j += 1
    
    return (out, delta, c_pair)
    


def closest_pair_1D(arr):  # assume len(arr) is power of 2
    if len(arr) == 2:
        if arr[0] < arr[1]:
            out = arr
            delta = np.abs(arr[0] - arr[1])
            c_pair = (arr[0], arr[1])
        
        elif arr[0] > arr[1]:
            out = [arr[1], arr[0]]
            delta = np.abs(arr[0] - arr[1])
            c_pair = (arr[0], arr[1])
            
        return (out, delta, c_pair)
    
    if len(arr) > 2:
        mid = len(arr) // 2
        lefthalf = arr[:mid]
        righthalf = arr[mid:]
        out_l, delta_l, c_pair_l = closest_pair_1D(lefthalf)
        out_r, delta_r, c_pair_r = closest_pair_1D(righthalf)
        out, delta_m, c_pair_split = closest_pair_split(out_l, out_r)
        
        if delta_l <= delta_r:
            if delta_l <= delta_m:
                delta = delta_l
                c_pair = c_pair_l
            else:
                delta = delta_m
                c_pair = c_pair_split
        
        else:
            if delta_r <= delta_m:
                delta = delta_r
                c_pair = c_pair_r
            else:
                delta = delta_m
                c_pair = c_pair_split
        
        return (out, delta, c_pair)

In [21]:
#closest_pair_1D([4,2])
closest_pair_1D([2,4,6,8,9,12,14,16])

([2, 4, 6, 8, 9, 12, 14, 16], 1, (8, 9))

In [14]:
# below is 2D
# assume (for convenience) all points have distinct x-coordinates, distinct y-coordinates(closest_pair_2D)
# point is tuple (..,..)
# out is list [...]
# c_distance is float
# c_pair is ((..,..), (..,..))

def distance(point1, point2): # assume point is a tuple
    distance = np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)
    return distance

In [15]:
def brute_force(arr): # arr is a list of tuples representing points, len(arr) <= 3
    out_x = [] # sorted array according to x-coordinates
    out_y = [] # sorted array according to y-coordinates
    c_dist = None
    c_pair = None
    
    for i in range(len(arr)):
        for j in range(i+1, len(arr)):
            d = distance(arr[i], arr[j])
            if c_dist == None or c_dist > d:
                c_dist = d
                c_pair = (arr[i], arr[j])
    
    out_x = sorted(arr)
    out_y = sorted(arr, key=lambda x:x[1])
    
    return (out_x, out_y, c_dist, c_pair)

In [ ]:
def closest_pair_2d(arr): # arr is a list of tuples representing points
    if len(arr) <= 3:
        out_x, out_y, d, pair = brute_force(arr)
        return (out_x, out_y, d, pair)
    
    elif len(arr) > 3:
        mid = len(arr) // 2
        lefthalf = arr[:mid]
        righthalf = arr[mid:]
        out_x_l, out_y_l, dl, pair_l = closest_pair_2d(lefthalf)
        out_x_r, out_y_r, dr, pair_r = closest_pair_2d(righthalf)
        
        out_x = sorted(out_x_l + out_y_l)
        out_y = sorted(out_y_l+out_y_r, key=lambda x:x[1])
        
        d = min(dl, dr)
        strip_points = []
        d_strip = None
        pair_strip = None
       
        mean_x = np.mean(list(zip(*(out_x)))[0])
        
        for point in out_y:
            if point[0] > mean_x - d and point[0] < mean_x + d:
                strip_points.append(point)
        
        for 
    
         